# Preppindata - Week 3

---

In [1]:
import pandas as pd
import numpy as np

## *Requirements*

### Input the data



In [2]:
trans = pd.read_csv('data/Week1_input.csv')
target = pd.read_csv('data/week3_Targets.csv')

#### A little inspection


In [3]:
trans.head()

,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date
0,DTB-716-679-576,1448,100001,2,20/03/2023 00:00:00
1,DS-795-814-303,7839,100001,2,15/11/2023 00:00:00
2,DSB-807-592-406,5520,100005,1,14/07/2023 00:00:00
3,DS-367-545-264,7957,100007,2,18/08/2023 00:00:00
4,DSB-474-374-857,5375,100000,2,26/08/2023 00:00:00


In [4]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Transaction Code     365 non-null    object
 1   Value                365 non-null    int64 
 2   Customer Code        365 non-null    int64 
 3   Online or In-Person  365 non-null    int64 
 4   Transaction Date     365 non-null    object
dtypes: int64(3), object(2)
memory usage: 14.4+ KB


In [5]:
target.head()

,Online or In-Person,Q1,Q2,Q3,Q4
0,Online,72500,70000,60000,60000
1,In-Person,75000,70000,70000,60000


In [6]:
target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Online or In-Person  2 non-null      object
 1   Q1                   2 non-null      int64 
 2   Q2                   2 non-null      int64 
 3   Q3                   2 non-null      int64 
 4   Q4                   2 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 208.0+ bytes


<br>

### For the transactions file:

#### - Filter the transactions to just look at DSB. These will be transactions that contain DSB in the Transaction Code field.

In [8]:
# Using a copy from the original dataset, to work on a new one
dsb_data = trans.loc[trans['Transaction Code'].str.contains('DSB')].copy()

In [9]:
# inspect
dsb_data.head()

,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date
2,DSB-807-592-406,5520,100005,1,14/07/2023 00:00:00
4,DSB-474-374-857,5375,100000,2,26/08/2023 00:00:00
5,DSB-448-546-348,4525,100009,1,27/05/2023 00:00:00
11,DSB-422-218-322,118,100010,1,12/05/2023 00:00:00
12,DSB-669-227-170,830,100001,1,15/04/2023 00:00:00


#### - Rename the values in the Online or In-person field, Online of the 1 values and In-Person for the 2 values

In [10]:
dsb_data['Online or In-Person'] = np.where(dsb_data['Online or In-Person'] == 1, 'Online', 'In-Person')

In [11]:
# Inspecting..
dsb_data.head()

,Transaction Code,Value,Customer Code,Online or In-Person,Transaction Date
2,DSB-807-592-406,5520,100005,Online,14/07/2023 00:00:00
4,DSB-474-374-857,5375,100000,In-Person,26/08/2023 00:00:00
5,DSB-448-546-348,4525,100009,Online,27/05/2023 00:00:00
11,DSB-422-218-322,118,100010,Online,12/05/2023 00:00:00
12,DSB-669-227-170,830,100001,Online,15/04/2023 00:00:00


#### - Change the date to be the quarter

In [12]:
# To ensure getting right quarters, is convennient to format date previously
dsb_data['Transaction Date'] = pd.to_datetime(dsb_data['Transaction Date'], format='%d/%m/%Y %H:%M:%S')

In [13]:
dsb_data['Quarter'] = dsb_data['Transaction Date'].dt.quarter

#### - Sum the transaction values for each quarter and for each Type of Transaction (Online or In-Person)



In [16]:
# Getting the Value per Quarter and Online...
# Dropping unused columns in the process...
dsb_data = dsb_data.groupby(['Quarter', 'Online or In-Person']).agg({'Value': 'sum'}).reset_index()

<br>

### For the targets file:

#### - Pivot the quarterly targets so we have a row for each Type of Transaction and each Quarter 

In [18]:
target_melt = target.melt('Online or In-Person').rename(columns={'variable':'Quarter'}).copy()

#### - Rename the fields

In [19]:
target_melt = target_melt.rename(columns={'value':'Quarterly Targets'})

#### - Remove the 'Q' from the quarter field and make the data type numeric 

In [20]:
target_melt['Quarter'] = target_melt['Quarter'].str[1]

In [21]:
# After comparing data types in both dataframes (target_melt and dsb_data), is needed to do a change:

target_melt['Quarter'] = target_melt['Quarter'].astype('int64')

In [22]:
target_melt

,Online or In-Person,Quarter,Quarterly Targets
0,Online,1,72500
1,In-Person,1,75000
2,Online,2,70000
3,In-Person,2,70000
4,Online,3,60000
5,In-Person,3,70000
6,Online,4,60000
7,In-Person,4,60000


In [23]:
dsb_data.head()

,Quarter,Online or In-Person,Value
0,1,In-Person,77576
1,1,Online,74562
2,2,In-Person,70634
3,2,Online,69325
4,3,In-Person,74189


### Join the two datasets together (You may need more than one join clause!)


In [26]:
output = dsb_data.merge(target_melt,on=['Online or In-Person','Quarter'])

In [27]:
output

,Quarter,Online or In-Person,Value,Quarterly Targets
0,1,In-Person,77576,75000
1,1,Online,74562,72500
2,2,In-Person,70634,70000
3,2,Online,69325,70000
4,3,In-Person,74189,70000
5,3,Online,59072,60000
6,4,In-Person,43223,60000
7,4,Online,61908,60000


### Calculate the Variance to Target for each row (help)


In [29]:
output['Variance to Targets'] = output['Value'] - output['Quarterly Targets']
output

,Quarter,Online or In-Person,Value,Quarterly Targets,Variance to Targets
0,1,In-Person,77576,75000,2576
1,1,Online,74562,72500,2062
2,2,In-Person,70634,70000,634
3,2,Online,69325,70000,-675
4,3,In-Person,74189,70000,4189
5,3,Online,59072,60000,-928
6,4,In-Person,43223,60000,-16777
7,4,Online,61908,60000,1908


### Output the data